## Getting input data from DB

In [1]:
import json
import requests

list_of_years = [2017, 2018, 2019, 2020, 2021]
list_of_qs = [1, 2, 3, 4]

dict_of_res = {}
for year in list_of_years:
    for q in list_of_qs:
        # Exclude 2017 quarter 1:
        if year == 2017 and q == 1:
            continue
        
        resp = json.loads(requests.get(f"http://localhost:8000/api/v1/input_data/{year}/{q}").text)
        if resp["code"] == 200:
            dict_of_res[str(year) + "_" + str(q)] = resp["data"]

## Get the fundamental data avg dict for all years and all quarters

In [2]:
dict_fund_data_avg_per_year_q = {}
for year_q, input_data_list in dict_of_res.items():
    dict_fund_data_avg_per_year_q[year_q] = input_data_list[0]["fundamental_data_avg"]

In [3]:
import pandas as pd

list_of_dicts_for_df = []
for year_q, dict_info in dict_fund_data_avg_per_year_q.items():
    # print(year_q)
    for filer_k, dict_kpis in dict_info.items():
        for kpi_k, dict_counts in dict_kpis.items():
            dict_row = {}
            # if kpi_k == 'roe':
            #     print(filer_k.upper())
            # print(f"{kpi_k.upper()} | all: {dict_counts['count_all']} | used: {dict_counts['count_used']} | median: {dict_counts['median']} | mean: {dict_counts['mean']} | not on time: {dict_counts.get('count_not_on_time', 0)} | used to all: {dict_counts['count_used']/dict_counts['count_all']}")
            # print(f"{kpi_k.upper()} | used to all: {round(dict_counts['count_used']/dict_counts['count_all'], 2)}")
            
            list_of_dicts_for_df.append({"year_q": year_q, "filer_type": filer_k, "kpi": kpi_k, "used_to_all": round(dict_counts['count_used']/dict_counts['count_all'], 2)})
            
    #     print('-----------------------------------')
    # print('-----------NEW ITERATION-----------')
    
df_for_plot = pd.DataFrame(list_of_dicts_for_df)

In [8]:
df_for_plot[(df_for_plot["year_q"] == "2019_3") & (df_for_plot["filer_type"] == "non_accelerated_filer")]

,year_q,filer_type,kpi,used_to_all
551,2019_3,non_accelerated_filer,roe,0.8
552,2019_3,non_accelerated_filer,roa,0.8
553,2019_3,non_accelerated_filer,roce,0.7
554,2019_3,non_accelerated_filer,gross_margin,0.8
555,2019_3,non_accelerated_filer,operating_profit_margin,0.8
556,2019_3,non_accelerated_filer,net_profit_margin,0.8
557,2019_3,non_accelerated_filer,debt_to_equity,0.7
558,2019_3,non_accelerated_filer,equity,0.8
559,2019_3,non_accelerated_filer,debt,0.6
560,2019_3,non_accelerated_filer,current,0.7


In [33]:
import plotly.express as px

fig = px.histogram(df_for_plot, x="used_to_all", color="kpi", marginal="rug",
                   hover_data=df_for_plot.columns, title="Percentage used KPIs from all for calculating mean/median | color: KPI")
fig.show()

In [35]:
import plotly.express as px

fig = px.histogram(df_for_plot, x="used_to_all", color="year_q", marginal="rug",
                   hover_data=df_for_plot.columns, title="Percentage used KPIs from all for calculating mean/median | color: year Q")
fig.show()

In [36]:
import plotly.express as px

fig = px.histogram(df_for_plot, x="used_to_all", color="filer_type", marginal="rug",
                   hover_data=df_for_plot.columns, title="Percentage used KPIs from all for calculating mean/median | color: Filer type")
fig.show()